In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import re
import os
import ast
import cv2
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import pickle
from folium.plugins import HeatMap
from mpl_toolkits.mplot3d import Axes3D
from datetime import date
from scipy import stats
from scipy.stats import norm,skew
from scipy.special import boxcox1p,inv_boxcox1p,boxcox
from sklearn.metrics import mean_squared_log_error, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder,StandardScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from collections import defaultdict
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

# CONSTANTS

In [3]:
RESTAURANT_NUMBER = 85
LIKE = 14
DISLIKE = -4

# DATASET

customer_id 86
gender
status_x
verified_x
location_number 90
location_type
latitude_x
longitude_x
id
latitude_y 95
longitude_y
vendor_category_id
delivery_charge
serving_distance
prepration_time 100
discount_percentage
rank
vendor_rating
device_type
location_number_obj 105
id_obj
target 107

In [4]:
train_data = []
val_data = []
test_data = []
train_labels = []
val_labels = []
test_labels = []
for i in range(RESTAURANT_NUMBER):
  cols = [0, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106]
  data = pd.read_csv("/content/drive/MyDrive/Colab/Bil573/train_data/restaurant_" + str(i) + ".csv")
  data.drop(data.columns[cols], axis=1, inplace=True)
  train_data.append(data[data.columns[0:RESTAURANT_NUMBER]])
  train_labels.append(data[data.columns[RESTAURANT_NUMBER]])
  data = pd.read_csv("/content/drive/MyDrive/Colab/Bil573/validation_data/restaurant_" + str(i) + ".csv")
  data.drop(data.columns[cols], axis=1, inplace=True)
  val_data.append(data[data.columns[0:RESTAURANT_NUMBER]])
  val_labels.append(data[data.columns[RESTAURANT_NUMBER]])
  data = pd.read_csv("/content/drive/MyDrive/Colab/Bil573/test_data/restaurant_" + str(i) + ".csv")
  cols = [0, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106]
  data.drop(data.columns[cols], axis=1, inplace=True)
  test_data.append(data[data.columns[0:RESTAURANT_NUMBER+1]])
  test_labels.append(data[data.columns[RESTAURANT_NUMBER+1]])

# TRAINING

In [5]:
# for n in range(1,16,2):
#   for m in range (RESTAURANT_NUMBER-1):
#     auc_best = 0
#     data_tr = train_data[m]
#     data_tr[data_tr==1] = LIKE
#     data_tr[data_tr==-1] = DISLIKE
#     data_vl = val_data[m]
#     data_vl[data_vl==1] = LIKE
#     data_vl[data_vl==-1] = DISLIKE
#     for i in range(n,31,2):
#       knn_model = KNeighborsClassifier(n_neighbors=i, weights='distance')
#       knn_model.fit(data_tr, train_labels[m])
#       pred = knn_model.predict(data_vl)
#       auc = roc_auc_score(val_labels[m], pred)
#       print(auc)
#       if auc > auc_best:
#         auc_best = auc
#       else:
#         print('Restaurant: {}\t n: {}\n'.format(m, i-2))
#         knn_model = KNeighborsClassifier(n_neighbors=i-2, weights='distance')
#         knn_model.fit(data_tr, train_labels[m])
#         pickle.dump(knn_model, open("/content/drive/MyDrive/Colab/Bil573/knn/knn_model" + str(m) + "_k" + str(n) + ".pkl", 'wb'))
#         break

# TEST

In [6]:
n = 1
preds = []
labels = []
ids = []
for m in range (RESTAURANT_NUMBER-1):
  knn_model = pickle.load(open("/content/drive/MyDrive/Colab/Bil573/knn/knn_model" + str(m) + "_k" + str(n) + ".pkl", 'rb'))
  data = test_data[m]
  data_te = data[data.columns[0:RESTAURANT_NUMBER]]
  data_te[data_te==1] = LIKE
  data_te[data_te==-1] = DISLIKE
  pred = knn_model.predict(data_te)
  preds = np.concatenate((preds, pred), axis=0)
  labels = np.concatenate((labels, test_labels[m]), axis=0)
  ids = np.concatenate((ids, data[data.columns[RESTAURANT_NUMBER]]), axis=0)

results = np.zeros((len(ids), 3))
results[:, 0] = ids
results[:, 1] = labels
results[:, 2] = preds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [7]:
for n in range(1,16,2):
  misses = np.zeros((35000, 1))
  for i in range(35000):
    customer = results[results[:, 0]==i]
    size = customer.shape
    if size[0] != 0:
      res = customer[customer[:, 2]==1.]
      size = res.shape
      if size[0] == 0:
        misses[i] = 1
      else:
        misses[i] = 0

  start = 0
  for m in range (RESTAURANT_NUMBER-1):
    knn_model = pickle.load(open("/content/drive/MyDrive/Colab/Bil573/knn/knn_model" + str(m) + "_k" + str(n) + ".pkl", 'rb'))
    data = test_data[m]
    data_te = data[data.columns[0:RESTAURANT_NUMBER]]
    data_te[data_te==1] = LIKE
    data_te[data_te==-1] = DISLIKE
    pred = knn_model.predict(data_te)

    for i in range(start, start+len(pred)):
      if misses[int(results[i, 0])]==1:
        results[i, 2] = pred[i-start]
    start += len(pred)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [8]:
columns = ['id','label', 'pred']
results = pd.DataFrame(data=results, columns=columns)

accs = []
accs2 = []
labels2 = []
accs3 = []
preds3 = []
for i in range(35000):
  customer = results[results['id']==i]
  size = customer.shape
  if size[0] != 0:
    result = customer[customer.columns[1:]]
    acc = accuracy_score(result['label'], result['pred'])
    accs.append(acc)

    res = result[result['pred']==1.]
    size = res.shape
    if size[0] != 0:
      labels2.append(res['label'])
      acc = accuracy_score(res['label'], res['pred'])
      accs2.append(acc)
    
    res = result[result['label']==1.]
    size = res.shape
    if size[0] != 0:
      preds3.append(res['pred'])
      acc = accuracy_score(res['label'], res['pred'])
      accs3.append(acc)

In [9]:
sum(accs)/len(accs)

0.8592349440708339

In [10]:
len(accs)

2855

In [11]:
sum(accs2)/len(accs2)
# önerilerin acc

0.392708692950356

In [12]:
hit_customer = 0
for i in range(len(labels2)):
  if sum(labels2[i]) > 0:
    hit_customer += 1

hit_customer/len(accs2)
# insan önerdiklerimden en az 1 ini beğendi mi

0.5176413726437893

In [13]:
len(accs2)

2069

In [14]:
hit_customer = 0
for i in range(len(preds3)):
  if sum(preds3[i]) > 0:
    hit_customer += 1

hit_customer/len(accs3)
# insanın beğenebileceklerinden en az 1 ini önerdim mi

0.41543832428238947

In [15]:
len(accs3)

2578